In [ ]:
import os
import sys

sys.path.insert(0, "/pscratch/sd/e/elynnwu/full-model/scripts/data_process")

from compute_ocean_dataset_e3sm import (
    construct_lazy_dataset,
    OceanDatasetConfig,
)

import xarray as xr
import numpy as np
import cftime
import xpartition
from dask.diagnostics import ProgressBar

In [ ]:
import dask
from dask.distributed import Client
import os

scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_file.json")

client = Client(scheduler_file=scheduler_file)
client

In [ ]:
config = OceanDatasetConfig.from_file("/pscratch/sd/e/elynnwu/full-model/scripts/data_process/configs/e3smv3-ocean-1deg.yaml").dataset_computation

In [ ]:
ds = construct_lazy_dataset(
    config=config
)

In [ ]:
outer_chunks = {"time": 10, "lon": 10, "lat": 10}

In [ ]:
ds = ds.chunk(outer_chunks)

In [ ]:
inner_chunks = {"time": 1, "lon": 10, "lat": 10}

In [ ]:
print(f"Output dataset size is {ds.nbytes / 1e9} GB")

In [ ]:
output_store = "/pscratch/sd/e/elynnwu/e3smv3-couple-run/zarrv3/2025-03-18-E3SM-piControl-ocean-ice/data.zarr"

In [ ]:
# this also has the same memory error
# delayed = ds.to_zarr(output_store, compute=False)
# dask.compute(delayed)

In [ ]:
ds.partition.initialize_store(output_store, inner_chunks=inner_chunks)
n_split = 50
for i in range(n_split):
    print(f"Writing segment {i + 1} / {n_split}")
    ds.partition.write(
        output_store,
        n_split,
        [config.standard_names.time_dim],
        i,
        collect_variable_writes=True,
    )